In [1]:
import torch
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import numpy as np
import wandb

In [2]:
!python3 -m wandb login eb7b1964fb84cd81de96b2a273ecf2bb6254aeac

wandb: Appending key for api.wandb.ai to your netrc file: /home/alexeyorlov53/.netrc


In [3]:
model_name = 'clefourrier/graphormer-base-pcqm4mv1'

### Upload Dataset

In [4]:
dataframe = pd.read_csv("data_100k_graph.csv")

In [ ]:
print('Percentage on NaNs:')
dataframe.isna().mean()

In [5]:
dataframe = dataframe.drop(columns=['Unnamed: 0', 'Smiles', 'ecfp1', 'ecfp2', 'ecfp3'])

In [6]:
def preprocess_array_column(df, column):
    for row in tqdm(range(len(df))):
        str_ints = eval(df.iloc[row][column])
        df.at[row, column] = str_ints

In [7]:
preprocess_array_column(dataframe, 'node_feat')
preprocess_array_column(dataframe, 'edge_index')
preprocess_array_column(dataframe, 'edge_attr')

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

In [21]:
dataframe['labels'] = dataframe['y'].apply(lambda x: [x])

In [24]:
dataframe = dataframe.drop(columns=['y'])

### Create Dataloader

In [29]:
from datasets import Dataset, DatasetDict

dataset = Dataset.from_pandas(dataframe)
train_testvalid = dataset.train_test_split(test_size=0.2, seed=15)

test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=15)

# 10% for test, 10 for validation, 80% for train
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

dataset

NameError: name 'dataframe' is not defined

In [30]:
from transformers.models.graphormer.collating_graphormer import preprocess_item, GraphormerDataCollator

dataset_processed = dataset.map(preprocess_item, batched=False)
# data_loader = GraphormerDataCollator(on_the_fly_processing=True) # либо препроцессинг либо коллайтор с on_the_fly_processing

NameError: name 'dataset' is not defined

In [29]:
dataset_processed.save_to_disk('dataset_graphormer_preprocessed')

Saving the dataset (0/158 shards):   0%|          | 0/80000 [00:00<?, ? examples/s]

Saving the dataset (0/21 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Saving the dataset (0/19 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [4]:
from datasets import load_from_disk
dataset_processed = load_from_disk('dataset_graphormer_preprocessed')

Loading dataset from disk:   0%|          | 0/158 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/21 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/19 [00:00<?, ?it/s]

### Create Model

In [6]:
# from transformers import AutoModel, AutoConfig

# config = AutoConfig.from_pretrained(model_name)
# AutoModel.from_pretrained(model_name, config=config)

In [6]:
# from transformers import AutoModel, AutoConfig

# class MolecularPropertiesRegression(torch.nn.Module):
#     def __init__(self, model_name, num_properties):
#         super(MolecularPropertiesRegression, self).__init__()
#         self.num_properties = num_properties

#         config = AutoConfig.from_pretrained(model_name)
#         self.transformer = AutoModel.from_pretrained(model_name, config=config)
#         # removing last layer of transformer
#         self.transformer.pooler = torch.nn.Identity()
#         # freezing transformer weights
#         for param in self.transformer.parameters():
#             param.requires_grad = False
#         self.regressor = torch.nn.Linear(768, num_properties)

#     def forward(self, input_ids = None, attention_mask=None):
#         outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        
#         last_hidden_state = outputs[0]
#         # last_hidden_state is the shape of (batch_size=32, input_sequence_length=512, hidden_size=768)
#         # so we take only hidden emdedding for [CLS] token (first) as it contains the entire context
#         # and would be sufficient for simple downstream tasks such as classification/regression
#         predicted_property_values = self.regressor(last_hidden_state[:, 0, : ].view(-1, 768))

#         return predicted_property_values
        

### Create DataLoader

In [5]:
from torch.utils.data import DataLoader
from transformers.models.graphormer.collating_graphormer import GraphormerDataCollator

data_collator = GraphormerDataCollator()

train_dataloader = DataLoader(
    dataset_processed['train'], shuffle = False, batch_size = 16, collate_fn = data_collator
)

eval_dataloader = DataLoader(
    dataset_processed['validation'], shuffle = False, batch_size = 16, collate_fn = data_collator
)

In [6]:
device = torch.device("cuda", index=3) if torch.cuda.is_available() else torch.device('cpu')

In [7]:
from transformers import GraphormerForGraphClassification

model = GraphormerForGraphClassification.from_pretrained(
    model_name, 
    num_classes=1,
    ignore_mismatched_sizes = True, # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
).to(device)

In [8]:
model

GraphormerForGraphClassification(
  (encoder): GraphormerModel(
    (graph_encoder): GraphormerGraphEncoder(
      (dropout_module): Dropout(p=0.0, inplace=False)
      (graph_node_feature): GraphormerGraphNodeFeature(
        (atom_encoder): Embedding(4609, 768, padding_idx=0)
        (in_degree_encoder): Embedding(512, 768, padding_idx=0)
        (out_degree_encoder): Embedding(512, 768, padding_idx=0)
        (graph_token): Embedding(1, 768)
      )
      (graph_attn_bias): GraphormerGraphAttnBias(
        (edge_encoder): Embedding(1537, 32, padding_idx=0)
        (edge_dis_encoder): Embedding(131072, 1)
        (spatial_pos_encoder): Embedding(512, 32, padding_idx=0)
        (graph_token_virtual_distance): Embedding(1, 32)
      )
      (emb_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-11): 12 x GraphormerGraphEncoderLayer(
          (dropout_module): Dropout(p=0.0, inplace=False)
          (activation_dropout_module): Dr

In [9]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epoch = 3

num_training_steps = num_epoch * len(train_dataloader)

lr_scheduler = get_scheduler(
    'linear',
    optimizer = optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps,
)

loss_func = torch.nn.MSELoss()

/home/alexeyorlov53/anaconda3/envs/test/lib/python3.11/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
wandb.init(
    project="graphormer",
    name="Graphormer Simple Classification on MolecularWeight",
    config={}
)

wandb: Currently logged in as: orlov-aleksei53 (moleculary-ai). Use `wandb login --relogin` to force relogin


### Training

In [11]:
from tqdm.auto import tqdm

progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(num_epoch * len(eval_dataloader)))

for epoch in range(num_epoch):
    model.train()
    epoch_loss = 0
    for batch in train_dataloader:
        input_batch = { k: v.to(device) for k, v in batch.items() }
        
        outputs = model(**input_batch)
        
        loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]
        loss.backward()
        epoch_loss += loss.item()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar_train.update(1)

    wandb.log({"loss/train": epoch_loss / len(train_dataloader)}, step=epoch)

    model.eval()
    epoch_loss = 0
    for batch in eval_dataloader:
        input_batch = { k: v.to(device) for k, v in batch.items() }

        with torch.no_grad():
            outputs = model(**input_batch)

        loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]
        epoch_loss += loss.item()
        
        progress_bar_eval.update(1)
    
    wandb.log({"loss/validation": epoch_loss / len(eval_dataloader)}, step=epoch)

  0%|          | 0/15000 [00:00<?, ?it/s]

  0%|          | 0/1875 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 390.00 MiB. GPU 3 has a total capacity of 39.39 GiB of which 131.31 MiB is free. Process 682795 has 3.72 GiB memory in use. Including non-PyTorch memory, this process has 35.53 GiB memory in use. Of the allocated memory 34.66 GiB is allocated by PyTorch, and 379.03 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
wandb.finish()

## Post Training Evaluation

In [28]:
test_dataloader = DataLoader(
    dataset_processed['test'], batch_size = 512, collate_fn = data_collator
)

model.eval()
epoch_loss = 0
for batch in tqdm(test_dataloader):
        input_batch = { k: v.to(device) for k, v in batch.items() if k in ['input_ids', 'attention_mask'] }
        labels_batch = { k: v.to(device) for k, v in batch.items() if k in molecular_properties }

        labeled_property_values = torch.stack(list(labels_batch.values())).T
        with torch.no_grad():
            predicted_property_values = model(**input_batch)

        loss = loss_func(predicted_property_values, labeled_property_values)
        epoch_loss += loss.item()
    
wandb.log({"loss/test": epoch_loss / len(test_dataloader)}, step=epoch)

  0%|          | 0/434 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
torch.onnx.export(model, images, "model.onnx")
wandb.save("model.onnx")

In [30]:
torch.save(model, model_name + '_and_linear_3epochs.pt')

In [ ]:
torch.cuda.empty_cache()